In [2]:
import sympy as sp
import numpy as np
from mpmath import *
from autograd import hessian
mp.dps = 20 # Setting the precision to 20-digit

# Loading the data

In [4]:
print('If possible, one should always work with 20-digit precision e.g. using the mpmath package.\nHowever, since the Python packages needed in order to compute the Hessian matrix are only\ncompatible with float64, we will have to tolerate roundoff errors.')
rhoinv = np.around(np.loadtxt('mu-e/L-chirality/uds/rhoinv_mueL_uds.txt'),20)
sigma = np.around(np.loadtxt('mu-e/L-chirality/uds/bounds_mueL_uds.txt'),20)

If possible, one should always work with 20-digit precision e.g. using the mpmath package.
However, since the Python packages needed in order to compute the Hessian matrix are only
compatible with float64, we will have to tolerate roundoff errors.


# Defining the model and the $\chi^2$

In [5]:
def my_model(c):
    # This defines the correlations that your UV-complete model
    # enforces on the Wilson coefficients.
    # It should take as input the free parameters of your model
    # and it should output the corresponding WC present in the fit
    # data you have loaded: 9 WC in our case (CuV,CdV,CuA,CdA,CsA,CuS,CdS,CsS,CuT)
    
    # We are going to consider an isovector model as an example
    
    p1,p2,p3 = c # c should be a list with your model's free parameters
    
    # Here we obtain the WC as a function of the input parameters
    CuV = p1
    CuA = p2
    CuS = p3
    CdV = -p1
    CdA = -p2
    CdS = -p3
    CuT = 0
    CsV,CsA,CsS = 0,0,0
    
    # Output should always have the same length as the dimension of rhoinv
    # and the ordering should match that of the paper
    return np.array([CuV,CdV,CuA,CdA,CsA,CuS,CdS,CsS,CuT])


# Definition of the chi2
# In principle this function should not require modification
def chi2(c):
    x = np.array([my_model(c)])/sigma
    return np.dot(x,np.dot(rhoinv,x.T))[0,0]

# Numerical estimation of the bounds with autograd

In [6]:
# Compute the hessian with autograd package
hess = hessian(chi2)(np.zeros(3))
bounds = np.sqrt(np.diag(np.linalg.inv(0.5*hess)))
print('95% C.L. bounds')
for i in range(len(bounds)):
    print('|p'+str(i+1)+'| < '+str(bounds[i]))

95% C.L. bounds
|p1| < 1.0325628010201314e-06
|p2| < 2.7507065896440325e-05
|p3| < 1.7998947239212554e-08


# Symbolic estimation of the bounds with sympy

In [7]:
c1 = sp.symbols('c1')
c2 = sp.symbols('c2')
c3 = sp.symbols('c3')
c = [c1,c2,c3]
ch2 = chi2(c)

hess = np.zeros((3,3))
for i in range(3):
    for j in range(i,3):
        hess[i,j] = sp.diff(ch2,c[i],c[j])
        hess[j,i] = hess[i,j]

bounds = np.sqrt(np.diag(np.linalg.inv(0.5*hess)))
print('95% C.L. bounds')
for i in range(len(bounds)):
    print('|p'+str(i+1)+'| < '+str(bounds[i]))

95% C.L. bounds
|p1| < 1.032562801020134e-06
|p2| < 2.7507065896440325e-05
|p3| < 1.7998947239212604e-08
